In [2]:
import imageio
from landlab import RasterModelGrid, imshow_grid
from landlab.components import DepressionFinderAndRouter, FlowAccumulator, LinearDiffuser, StreamPowerEroder
from matplotlib import pyplot as plt
import numpy as np
import os
import rasterio
from rasterio.plot import show
import re

In [ ]:
mg: RasterModelGrid
with rasterio.open(r'..\src\SanFernando.tif') as f:
    z_grd = f.read(1)
    z_grd[z_grd == -9999] = 0

cell_size = f.transform.a
dem_width = f.width
dem_height = f.height

mg = RasterModelGrid((dem_width, dem_height), cell_size, (f.bounds[0], f.bounds[1]))
zr = mg.add_zeros('topographic__elevation', at = 'node', clobber = True)
zr += z_grd[::-1, :].ravel(order = 'F')

mg.imshow('topographic__elevation', cmap='terrain')

def show_plot_2d(val_grd: np.ndarray,
                 z_label = 'Elevation (m)',
                 title = 'DEM',
                 cmap = 'terrain'):
    fig, ax = plt.subplots()
    im = ax.imshow(val_grd, cmap = cmap)
    cb = fig.colorbar(im, ax = ax)
    cb.set_label(z_label)
    ax.set(title = title, xlabel = 'X', ylabel = 'Y')
    plt.show()

show_plot_2d(z_grd)


In [ ]:
# Terrain parameters
k_hs = 0.05         # m^2/yr
k_sp = 1e-5         # Depending on m_sp and n_sp
m_sp = 0.5          # Exponent on drainage area in Stream Power equation
n_sp = 1            # Exponent on slope in Stream Power equation
uplift_rate = 0.001 # m/yr

dt = 10000          # Time step, yr
tmax = 1e6          # Total model run time, yr
t_series = np.arange(0, tmax, dt)
total_time = 0

In [ ]:
fr = FlowAccumulator(mg)
spr = StreamPowerEroder( mg, K_sp=k_sp, m_sp=m_sp, n_sp=n_sp, threshold_sp=0.0)
dfn = LinearDiffuser(mg, linear_diffusivity=k_hs, deposit=False)
df = DepressionFinderAndRouter(mg)

In [ ]:
if not os.path.exists('result'):
    os.makedirs('result')

for time in t_series:
    zr[mg.core_nodes] += uplift_rate * dt
    dfn.run_one_step(dt)
    fr.run_one_step()
    df.map_depressions()
    spr.run_one_step(dt)
    total_time += dt
    print(f'\r{total_time}', end = '')

    original_array= mg.at_node["topographic__elevation"]
    reshaped_array = original_array.reshape(dem_width, dem_height)
    mg.imshow("topographic__elevation", cmap="terrain")

    title_text = f"$k_{{sp}}$={k_sp};$k_{{hs}}$={k_hs};$time$={total_time};$dx$={cell_size}"
    plt.title(title_text)
    plt.savefig(os.path.join('result', f'figure{time:.0f}.png'))
    plt.close()

In [3]:
def create_gif_from_folder(folder_path, output_filename, fps = 10):
    images = []
    filtered = []

    def getNumber(s):
        for i, match in enumerate(re.finditer(r'(?<=figure)\d+(?=\.png)', s), start = 1):
            return int(match.group())

    for i in os.listdir(folder_path):
        filtered.append(getNumber(i))

    file_paths = [os.path.join(folder_path, f'figure{i}.png') for i in sorted(filtered)]
    
    # Read each file and append to images list
    for file_path in file_paths:
        images.append(imageio.imread(file_path))
    
    # Save the images as a GIF
    imageio.mimsave(output_filename, images, fps = fps)

# Specify the folder and the output GIF file name
folder_path = 'result'
output_gif_filename = 'output.gif'

# Create the GIF
create_gif_from_folder(folder_path, output_gif_filename)

C:\Users\DanielHo\AppData\Local\Temp\ipykernel_732\1476577973.py:16: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))
